## Análise de propostas de fornecedores

**Contexto**

Avaliando as propostas de 8 fornecedores, pegando o valor mínimo de cada item para chegar em um valor 'ótimo', para então conseguir uma possivel redução de valores e baixar a proposta final deles

In [1]:
#Bibliotecas necessárias

from zipfile import ZipFile
import os
import pandas as pd


In [2]:
# Código para fazer o download e a extração dos arquivos necessários

!wget  -q https://github.com/LeonnyGR/Lista_De_Fornecedores/archive/refs/heads/main.zip

In [3]:
# Descompactando os arquivos

with ZipFile('main.zip', 'r') as zipObj:
  zipObj.extractall()

In [4]:
# Localizando e armazenando os arquivos em uma variável

arquivos = os.listdir('/kaggle/working/Lista_De_Fornecedores-main')
list.sort(arquivos)
print(arquivos)

['Empresa A.xlsx', 'Empresa B.xlsx', 'Empresa C.xlsx', 'Empresa D.xlsx', 'Empresa E.xlsx', 'Empresa F.xlsx', 'Empresa G.xlsx', 'Empresa H.xlsx']


In [5]:
# Lendo a base de dados, renomeando a coluna 'Total' para o nome de cada empresa e criando uma tabela única que contenha o valor total delas

licitacao = pd.read_excel('/kaggle/working/Lista_De_Fornecedores-main/Empresa A.xlsx')
licitacao = licitacao.rename(columns={'Total': 'Empresa A'})
licitacao = licitacao.drop(['Valor Unitário'], axis=1)

empresas = arquivos[1:]

for empresa in empresas:
  proposta = pd.read_excel('/kaggle/working/Lista_De_Fornecedores-main/' + empresa)
  licitacao[empresa[:-5]] = proposta['Total'].copy()

licitacao.head()



,Item,Descrição do Item,Quantidade,Empresa A,Empresa B,Empresa C,Empresa D,Empresa E,Empresa F,Empresa G,Empresa H
0,1,Item 1,50,2350,1250,1450,2100,2050,1000,650,2450
1,2,Item 2,96,2688,1632,2496,2208,1248,2592,1440,1056
2,3,Item 3,58,1566,754,1276,1914,1508,2146,2378,1508
3,4,Item 4,69,1863,828,3450,759,1932,2208,1725,3105
4,5,Item 5,61,1342,1281,1708,1769,2684,1830,2318,610


## Verificando Valores Totais

In [6]:
#Somando o valor total de cada empresa

for coluna in licitacao.columns[3:]:
  total = licitacao[coluna].sum()
  print('{}: R$ {:,.2f}'.format(coluna, total))

Empresa A: R$ 429,316.00
Empresa B: R$ 267,971.00
Empresa C: R$ 447,536.00
Empresa D: R$ 421,243.00
Empresa E: R$ 446,904.00
Empresa F: R$ 472,515.00
Empresa G: R$ 428,703.00
Empresa H: R$ 459,468.00


**Insight:** Podemos ver que a empresa B possui os menores valores, são valores muito discrepantes, temos que nos atentar a esse fato, pois o fornecedor pode ter feito uma confusão e não ter entendido nossa proposta ou realmente ele possui preços mais baixos.

## Resultado "ótimo":

In [7]:
# Criando uma coluna com o valor minimo de cada Item comparando todas as empresas

licitacao['Mínimo'] = licitacao.drop(['Item','Descrição do Item', 'Quantidade'],axis=1).min(axis=1)
display(licitacao)

minimo = licitacao['Mínimo'].sum()
print(minimo)

,Item,Descrição do Item,Quantidade,Empresa A,Empresa B,Empresa C,Empresa D,Empresa E,Empresa F,Empresa G,Empresa H,Mínimo
0,1,Item 1,50,2350,1250,1450,2100,2050,1000,650,2450,650
1,2,Item 2,96,2688,1632,2496,2208,1248,2592,1440,1056,1056
2,3,Item 3,58,1566,754,1276,1914,1508,2146,2378,1508,754
3,4,Item 4,69,1863,828,3450,759,1932,2208,1725,3105,759
4,5,Item 5,61,1342,1281,1708,1769,2684,1830,2318,610,610
...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,Item 196,60,1440,1440,1020,2400,1620,2100,2160,2580,1020
196,197,Item 197,82,2050,1066,3116,1886,1394,3608,3608,1804,1066
197,198,Item 198,100,3400,2000,1100,1000,3300,4900,1100,1100,1000
198,199,Item 199,67,804,1072,2680,1809,2814,2881,670,3015,670


193299


## Criando uma planilha de desvios por fornecedor:

In [8]:
# Criando uma planilha para cada fornecedor, pra ele saber o quão longe ele está do valor mínimo dos itens

for empresa in licitacao.columns[3:-1]:
    desvio = licitacao[['Item','Descrição do Item', 'Quantidade',empresa,'Mínimo']]
    desvio['Delta'] = desvio[empresa] - desvio['Mínimo']
    desvio.to_excel('Desvio - ' + empresa + '.xlsx', index = False)

/tmp/ipykernel_20/521148253.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  desvio['Delta'] = desvio[empresa] - desvio['Mínimo']
/tmp/ipykernel_20/521148253.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  desvio['Delta'] = desvio[empresa] - desvio['Mínimo']
/tmp/ipykernel_20/521148253.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

**Insight 2:** Criamos uma planilha para cada fornecedor analisar o preço sugerido de cada item. A ideia é que eles reduzam os valores dos seus itens e cheguem num valor ideal para fechar com a nossa empresa